In [ ]:
# pip install hypopt 

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from hypopt import GridSearch

In [2]:
train = pd.read_csv('preprocessed_train.csv')
test = pd.read_csv('preprocessed_test.csv')

# remove last event for each case in order to not break the model when accessing the next event time + make predictions on seconds_next + do tuning (grid search, feature selection) and cv


# Removing the last event from each case to reduce noise in the models

In [3]:
train.drop(train.groupby('case concept:name').tail(1).index, axis=0, inplace=True)
test.drop(test.groupby('case concept:name').tail(1).index, axis=0, inplace=True)

# Evaluation


In [4]:
def time_evaluation(y_test, y_pred, model: str):
 
    print(f"Error metrics (measured in hours) for the {model} when predicting the next event's Unix timestamp")
    print('\n')
    print('Mean Absolute Error:', round(mean_absolute_error(y_test, y_pred)/3600,3))
    print('Root Mean Squared Error:', round(np.sqrt(mean_squared_error(y_test, y_pred)/3600),3))
    print('R2 score:', round(r2_score(y_test, y_pred),3))
    
    
def event_evaluation(y_test, y_pred, model: str, avg="weighted"):

    precision = precision_score(y_test, y_pred, average=avg, zero_division=0)
    recall = recall_score(y_test, y_pred, average=avg, zero_division=0)
    F1_score = f1_score(y_test, y_pred, average=avg, zero_division=0)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f'Error metrics for the {model} when predicting the next event')
    print('\n')
    print(f'Accuracy: {round(accuracy,3)}.')
    print(f'Precision: {round(precision,3)}')
    print(f'Recall: {round(recall,3)}')
    print(f'f1-score: {round(F1_score,3)}')
#     print(confusion_matrix(y_test, y_pred))

# Data splitting and encoding

In [5]:
def make_val_set(dataframe):
    """make a validation set from the dataframe"""
    
    #set seed for reproducibility
    np.random.seed(69)
    
    #extract all unique case IDs
    unique_ids = dataframe['case concept:name'].unique()
    
    #select 10% of the unique IDs and use them to create a validation set
    samples = np.random.choice(unique_ids, size=int(len(unique_ids)*0.1), replace=False)
    val_set = dataframe[dataframe['case concept:name'].isin(samples)]
    
    train = dataframe[~dataframe['case concept:name'].isin(samples)]
    
    return val_set, train

In [6]:
train_LE = train.copy()
train_LE = train_LE.replace({'event lifecycle:transition': {'SCHEDULE': 0, 'START': 1, 'COMPLETE': 2}})

test_LE = test.copy()
test_LE = test_LE.replace({'event lifecycle:transition': {'SCHEDULE': 0, 'START': 1, 'COMPLETE': 2}})


train_OHE = pd.get_dummies(train_LE, prefix = ['current', 'prev', '2prev'], 
                           columns = ['event concept:name', 'prev_event', '2prev_event'])
test_OHE = pd.get_dummies(test_LE, prefix = ['current', 'prev', '2prev'], 
                          columns = ['event concept:name', 'prev_event', '2prev_event'])

# train_OHE = pd.get_dummies(train_OHE, prefix=['type'], columns = ['prev_event'])
# test_OHE = pd.get_dummies(test_OHE, prefix=['type'], columns = ['prev_event'])

val_OHE, train_OHE = make_val_set(train_OHE)

In [7]:
train_OHE.shape, val_OHE.shape, test_OHE.shape, train_LE.shape, test_LE.shape

((146106, 95), (17148, 95), (49291, 91), (163254, 26), (49291, 26))

In [8]:
temp3 = []
for element in list(train_OHE.columns):
    if element not in list(test_OHE.columns):
        temp3.append(element)
print('events that happen in the train but not the test set: ' + str(temp3))

events that happen in the train but not the test set: ['current_W_Wijzigen contractgegevens', 'prev_W_Wijzigen contractgegevens', '2prev_A_CANCELLED', '2prev_W_Wijzigen contractgegevens']


In [9]:
features_time = ['timestamp_finish', 'seconds_prev', 'active cases', 'day_week', 'time_of_day', 
                 'case AMOUNT_REQ', 'event lifecycle:transition', 
       'current_A_ACCEPTED', 'current_A_ACTIVATED', 'current_A_APPROVED',
       'current_A_CANCELLED', 'current_A_DECLINED', 'current_A_FINALIZED',
       'current_A_PARTLYSUBMITTED', 'current_A_PREACCEPTED',
       'current_A_REGISTERED', 'current_A_SUBMITTED', 'current_O_ACCEPTED',
       'current_O_CANCELLED', 'current_O_CREATED', 'current_O_DECLINED',
       'current_O_SELECTED', 'current_O_SENT', 'current_O_SENT_BACK',
       'current_W_Afhandelen leads', 'current_W_Beoordelen fraude',
       'current_W_Completeren aanvraag',
       'current_W_Nabellen incomplete dossiers', 'current_W_Nabellen offertes',
       'current_W_Valideren aanvraag', 'current_W_Wijzigen contractgegevens']

features_time_test = ['timestamp_finish', 'seconds_prev', 'active cases', 'day_week', 'time_of_day', 
                 'case AMOUNT_REQ', 'event lifecycle:transition', 
       'current_A_ACCEPTED', 'current_A_ACTIVATED', 'current_A_APPROVED',
       'current_A_CANCELLED', 'current_A_DECLINED', 'current_A_FINALIZED',
       'current_A_PARTLYSUBMITTED', 'current_A_PREACCEPTED',
       'current_A_REGISTERED', 'current_A_SUBMITTED', 'current_O_ACCEPTED',
       'current_O_CANCELLED', 'current_O_CREATED', 'current_O_DECLINED',
       'current_O_SELECTED', 'current_O_SENT', 'current_O_SENT_BACK',
       'current_W_Afhandelen leads', 'current_W_Beoordelen fraude',
       'current_W_Completeren aanvraag',
       'current_W_Nabellen incomplete dossiers', 'current_W_Nabellen offertes',
       'current_W_Valideren aanvraag']

target_time = 'seconds_next'

features_event = ['active cases', 'day_week', 'time_of_day',  
                  'event lifecycle:transition', 'case AMOUNT_REQ', 
       'current_A_ACCEPTED', 'current_A_ACTIVATED', 'current_A_APPROVED',
       'current_A_CANCELLED', 'current_A_DECLINED', 'current_A_FINALIZED',
       'current_A_PARTLYSUBMITTED', 'current_A_PREACCEPTED',
       'current_A_REGISTERED', 'current_A_SUBMITTED', 'current_O_ACCEPTED',
       'current_O_CANCELLED', 'current_O_CREATED', 'current_O_DECLINED',
       'current_O_SELECTED', 'current_O_SENT', 'current_O_SENT_BACK',
       'current_W_Afhandelen leads', 'current_W_Beoordelen fraude',
       'current_W_Completeren aanvraag',
       'current_W_Nabellen incomplete dossiers', 'current_W_Nabellen offertes',
       'current_W_Valideren aanvraag', 'current_W_Wijzigen contractgegevens',
       'prev_A_ACCEPTED', 'prev_A_ACTIVATED', 'prev_A_APPROVED',
       'prev_A_CANCELLED', 'prev_A_DECLINED', 'prev_A_FINALIZED',
       'prev_A_PARTLYSUBMITTED', 'prev_A_PREACCEPTED', 'prev_A_REGISTERED',
       'prev_A_SUBMITTED', 'prev_FIRST EVENT', 'prev_O_ACCEPTED',
       'prev_O_CANCELLED', 'prev_O_CREATED', 'prev_O_DECLINED',
       'prev_O_SELECTED', 'prev_O_SENT', 'prev_O_SENT_BACK',
       'prev_W_Afhandelen leads', 'prev_W_Beoordelen fraude',
       'prev_W_Completeren aanvraag', 'prev_W_Nabellen incomplete dossiers',
       'prev_W_Nabellen offertes', 'prev_W_Valideren aanvraag',
       'prev_W_Wijzigen contractgegevens', '2prev_A_ACCEPTED',
       '2prev_A_ACTIVATED', '2prev_A_APPROVED', '2prev_A_CANCELLED',
       '2prev_A_FINALIZED', '2prev_A_PARTLYSUBMITTED', '2prev_A_PREACCEPTED',
       '2prev_A_REGISTERED', '2prev_A_SUBMITTED', '2prev_FIRST EVENT',
       '2prev_O_ACCEPTED', '2prev_O_CANCELLED', '2prev_O_CREATED',
       '2prev_O_SELECTED', '2prev_O_SENT', '2prev_O_SENT_BACK',
       '2prev_W_Afhandelen leads', '2prev_W_Beoordelen fraude',
       '2prev_W_Completeren aanvraag', '2prev_W_Nabellen incomplete dossiers',
       '2prev_W_Nabellen offertes', '2prev_W_Valideren aanvraag',
       '2prev_W_Wijzigen contractgegevens']

features_event_test = ['active cases', 'day_week', 'time_of_day',  
                  'event lifecycle:transition', 'case AMOUNT_REQ', 
       'current_A_ACCEPTED', 'current_A_ACTIVATED', 'current_A_APPROVED',
       'current_A_CANCELLED', 'current_A_DECLINED', 'current_A_FINALIZED',
       'current_A_PARTLYSUBMITTED', 'current_A_PREACCEPTED',
       'current_A_REGISTERED', 'current_A_SUBMITTED', 'current_O_ACCEPTED',
       'current_O_CANCELLED', 'current_O_CREATED', 'current_O_DECLINED',
       'current_O_SELECTED', 'current_O_SENT', 'current_O_SENT_BACK',
       'current_W_Afhandelen leads', 'current_W_Beoordelen fraude',
       'current_W_Completeren aanvraag',
       'current_W_Nabellen incomplete dossiers', 'current_W_Nabellen offertes',
       'current_W_Valideren aanvraag',
       'prev_A_ACCEPTED', 'prev_A_ACTIVATED', 'prev_A_APPROVED',
       'prev_A_CANCELLED', 'prev_A_DECLINED', 'prev_A_FINALIZED',
       'prev_A_PARTLYSUBMITTED', 'prev_A_PREACCEPTED', 'prev_A_REGISTERED',
       'prev_A_SUBMITTED', 'prev_FIRST EVENT', 'prev_O_ACCEPTED',
       'prev_O_CANCELLED', 'prev_O_CREATED', 'prev_O_DECLINED',
       'prev_O_SELECTED', 'prev_O_SENT', 'prev_O_SENT_BACK',
       'prev_W_Afhandelen leads', 'prev_W_Beoordelen fraude',
       'prev_W_Completeren aanvraag', 'prev_W_Nabellen incomplete dossiers',
       'prev_W_Nabellen offertes', 'prev_W_Valideren aanvraag',
       '2prev_A_ACCEPTED', '2prev_A_ACTIVATED', '2prev_A_APPROVED',
       '2prev_A_FINALIZED', '2prev_A_PARTLYSUBMITTED', '2prev_A_PREACCEPTED',
       '2prev_A_REGISTERED', '2prev_A_SUBMITTED', '2prev_FIRST EVENT',
       '2prev_O_ACCEPTED', '2prev_O_CANCELLED', '2prev_O_CREATED',
       '2prev_O_SELECTED', '2prev_O_SENT', '2prev_O_SENT_BACK',
       '2prev_W_Afhandelen leads', '2prev_W_Beoordelen fraude',
       '2prev_W_Completeren aanvraag', '2prev_W_Nabellen incomplete dossiers',
       '2prev_W_Nabellen offertes', '2prev_W_Valideren aanvraag']

target_event = 'next_event'


# TIME
X_train_time = train_OHE[features_time]
y_train_time = train_OHE[target_time]

X_val_time = val_OHE[features_time]
y_val_time = val_OHE[target_time]

X_test_time = test_OHE[features_time_test].copy()
y_test_time = test_OHE[target_time]


# EVENT
X_train_event = train_OHE[features_event]
y_train_event = train_OHE[target_event]

X_val_event = val_OHE[features_event]
y_val_event = val_OHE[target_event]

X_test_event = test_OHE[features_event_test].copy()
y_test_event = test_OHE[target_event]



X_test_time['current_W_Wijzigen contractgegevens']=0
X_test_event[['current_W_Wijzigen contractgegevens', 'prev_W_Wijzigen contractgegevens', '2prev_A_CANCELLED', '2prev_W_Wijzigen contractgegevens']]=0

In [10]:
X_train_event.shape, X_train_time.shape, X_test_event.shape, X_test_time.shape

((146106, 77), (146106, 31), (49291, 77), (49291, 31))

In [11]:
# ohe = OneHotEncoder().fit(train['event concept:name'].to_numpy().reshape(-1, 1))

# transformed = ohe.transform(train['event concept:name'].to_numpy().reshape(-1, 1))
# train_OHE = pd.DataFrame(transformed, columns=jobs_encoder.get_feature_names())

# Event prediction

In [60]:
def RandomForestEvents(X_train, y_train, X_val, y_val):
    
    
    params={'max_depth': [32,64],
     'min_samples_leaf': [16,32],
     'n_estimators': [20,300,400]}

    
    forest_clf = RandomForestClassifier(bootstrap=True, criterion='gini', max_features='sqrt', random_state=42)    
    
    grid = GridSearch(model=forest_clf, param_grid=params, parallelize=False)

    grid.fit(X_train, y_train, X_val, y_val)
    
    return grid.best_estimator_, grid.best_estimator_.score(X_train, y_train), grid.best_estimator_.score(X_val, y_val)


In [61]:
rf_event, rf_event_train, rf_event_val = RandomForestEvents(X_train_event, y_train_event, X_val_event, y_val_event)

In [62]:
rf_event.get_params(),rf_event_train, rf_event_val, rf_event.score(X_test_event, y_test_event)

({'bootstrap': True,
  'ccp_alpha': 0.0,
  'class_weight': None,
  'criterion': 'gini',
  'max_depth': 64,
  'max_features': 'sqrt',
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 32,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 400,
  'n_jobs': None,
  'oob_score': False,
  'random_state': 0,
  'verbose': 0,
  'warm_start': False},
 0.8273992854502895,
 0.828318171215302,
 0.41857539916009007)

In [ ]:
# forest_clf = RandomForestClassifier(bootstrap= True,
#   ccp_alpha= 0.0,
#   class_weight= None,
#   criterion= 'gini',
#   max_depth= 10,
#   max_features= 'sqrt',
#   max_leaf_nodes= None,
#   max_samples= None,
#   min_impurity_decrease= 0.0,
#   min_impurity_split= None,
#   min_samples_leaf= 8,
#   min_samples_split= 2,
#   min_weight_fraction_leaf= 0.0,
#   n_estimators= 100,
#   n_jobs= None,
#   oob_score= False,
#   random_state= 0,
#   verbose= 0,
#   warm_start= False)  

# forest_clf.fit(X_train_event, y_train_event)

# forest_clf_train = forest_clf.predict(X_train_event)
# forest_clf_val = forest_clf.predict(X_val_event)
# forest_clf_test = forest_clf.predict(X_test_event)


In [ ]:
# forest_clf_train
# forest_clf_val
# forest_clf_test

# Time prediction

In [48]:
def RandomForestTime(X_train, X_test, y_train, y_test):
    '''no grid search, no cv, or feature selection, do for final model
    '''
    
    X_train_time = X_train[['event concept:name','prev_event', '2prev_event', 'seconds_prev', 'timestamp_finish']]
    y_train_time = y_train['seconds_next']



    X_test_time = X_test[['event concept:name','prev_event', '2prev_event', 'seconds_prev', 'timestamp_finish']]
    y_test_time = y_test['seconds_next']
    
    forest_reg = RandomForestRegressor(n_estimators=10, bootstrap = False, random_state=42)
    forest_reg.fit(X_train_time, y_train_time)
    
    y_pred = forest_reg.predict(X_test_time)

    return forest_reg, y_pred

def RandomForestEvents(X_train, y_train, X_val, y_val):
    
    
    params={'max_depth': [32,64],
     'min_samples_leaf': [16,32],
     'n_estimators': [60,250,300]}

    forest_reg = RandomForestRegressor(bootstrap = False, max_features='sqrt', random_state=42)
    
    grid = GridSearch(model=forest_reg, param_grid=params, parallelize=False)

    grid.fit(X_train, y_train, X_val, y_val)
    
    return grid.best_estimator_, grid.best_estimator_.score(X_train, y_train), grid.best_estimator_.score(X_val, y_val)


In [49]:
rf_time, rf_time_train, rf_time_val = RandomForestEvents(X_train_time, y_train_time, X_val_time, y_val_time)

In [50]:
rf_time.get_params(),rf_time_train, rf_time_val, rf_time.score(X_test_time, y_test_time)

({'bootstrap': False,
  'ccp_alpha': 0.0,
  'criterion': 'mse',
  'max_depth': 64,
  'max_features': 'sqrt',
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 32,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 300,
  'n_jobs': None,
  'oob_score': False,
  'random_state': 0,
  'verbose': 0,
  'warm_start': False},
 0.3558461206864064,
 0.320762015151943,
 0.31357795870005545)

In [52]:
time_evaluation(y_test_time, rf_time.predict(X_test_time), 'rf time')

Error metrics (measured in hours) for the rf time when predicting the next event's Unix timestamp


Mean Absolute Error: 9.597
Root Mean Squared Error: 1818.293
R2 score: 0.314


In [63]:
event_evaluation(y_test_event, rf_event.predict(X_test_event), 'rf event')

Error metrics for the rf event when predicting the next event


Accuracy: 0.419.
Precision: 0.364
Recall: 0.419
f1-score: 0.372


# Decision Tree time prediction

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def DecisionTreeTime(X_train, X_test, y_train, y_test):
    '''no grid search, no cv, or feature selection, do for final model
    '''
    
    X_train_time = X_train[['event concept:name','prev_event', '2prev_event', 'seconds_prev', 'timestamp_finish']]
    y_train_time = y_train['seconds_next']

    X_test_time = X_test[['event concept:name','prev_event', '2prev_event', 'seconds_prev', 'timestamp_finish']]
    y_test_time = y_test['seconds_next']
    
    dt_reg = DecisionTreeRegressor(random_state=42)
    dt_reg.fit(X_train_time, y_train_time)
    
    y_pred = dt_reg.predict(X_test_time)

    return dt_reg, y_pred

dt_time, y_pred_dt_time = DecisionTreeTime(X_train, X_test, y_train, y_test)
time_evaluation(y_test['seconds_next'], y_pred_dt_time, 'Decision tree regressor')


In [ ]:
# get feature importances and their names
feature_importances_dt_time = dt_time.feature_importances_
feature_names_dt_time = ['seconds_prev', 'timestamp_finish', 'prev_event', 'event concept:name', '2prev_event']

# sort the features by importance in descending order
inx = np.argsort(feature_importances_dt_time)[::-1]
sorted_imp_dt_time = feature_importances_dt_time[inx]

plt.figure(figsize=(10, 8))
plt.bar(feature_names_dt_time, sorted_imp_dt_time, color=['brown', 'green', 'blue', 'yellow'])
plt.xticks(rotation=10)
plt.title('Feature Importances of Decision tree time estimator')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()

## Time, Memory and CPU usage

### Memory and CPU usage

In [ ]:
import psutil
import time

# start measuring CPU and memory usage
process = psutil.Process()

# start point
start_time = time.time()

forest_event, y_pred_event = RandomForestEvents(X_train, X_test, y_train, y_test)
event_evaluation(y_test['next_event'], y_pred_event, 'Random forest classifier')
# forest_time, y_pred_time = RandomForestTime(X_train, X_test, y_train, y_test)
# time_evaluation(y_test['next_timestamp'], y_pred_time, 'Random forest regressor')
# end point
end_time = time.time()

# calculate time taken
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

# measure CPU and memory usage
cpu_usage = process.cpu_percent()
memory_usage = process.memory_info().rss / 1024 / 1024  # in MB
print(f"CPU usage: {cpu_usage:.2f}%")
print(f"Memory usage: {memory_usage:.2f} MB")

### Time usage

In [ ]:
start_time = time.time()

# RandomForestEvents function
forest_event, y_pred_event = RandomForestEvents(X_train, X_test, y_train, y_test)

# event_evaluation function
event_evaluation(y_test['next_event'], y_pred_event, 'Random forest classifier')

print(f"Time taken by RandomForestEvents: {time.time() - start_time} seconds")

start_time = time.time()

# RandomForestTime function
forest_time, y_pred_time = RandomForestTime(X_train, X_test, y_train, y_test)

# # time_evaluation function
# time_evaluation(y_test['next_timestamp'], y_pred_time, 'Random forest regressor')

print(f"Time taken by RandomForestTime: {time.time() - start_time} seconds")

# Visualizations

### Confusion matrix for Random Forest estimator

In [ ]:
cm = confusion_matrix(y_test['next_event'], y_pred_event, normalize='true')

# plot confusion matrix as a heatmap
sns.set(font_scale=1.2)
fig, ax = plt.subplots(figsize=(10,8), facecolor='w')  # add facecolor attribute
sns.heatmap(cm, fmt='g', cmap='RdBu_r', ax=ax, annot_kws={"size": 14})
ax.set_xlabel('Predicted labels', fontsize=14)
ax.set_ylabel('True labels', fontsize=14)
ax.set_title('Confusion matrix for Random Forest estimator', fontsize=16)
plt.show()

### Naive and Random Forest estimators accuracy and precision

In [ ]:
# plot the naive and random forest f1, acc, pre, recall
metrics_df = pd.DataFrame({
    ' ': ['F1 Score', 'Accuracy', 'Precision', 'Recall'],
    'Naive': ['0.266', '0.346', '0.297', '0.346'],
    'Random Forest': ['0.748', '0.785', '0.772', '0.785']    
})

metrics_df.to_string(index=False)
metrics_df

### Bar chart for feature importance of Random Forest event estimator

In [ ]:
import matplotlib.pyplot as plt

# get feature importances and their names
importances = forest_event.feature_importances_
# feature_names = X_train.columns.values
feature_names = ['Current event', 'Previous event', '2nd previous event']

# sort the features by importance in descending order
indices = np.argsort(importances)[::-1]
sorted_importances = importances[indices]

plt.figure(figsize=(10,6))
plt.bar(feature_names, sorted_importances, color=['brown', 'green', 'blue'])
plt.title('Feature Importances of Random Forest event estimator')
plt.xlabel('Features')
plt.xticks(rotation=0)
plt.ylabel('Importance')
plt.show()


### Bar chart feature importance Random Forest time estimator

In [ ]:
# get feature importances and their names
importances = forest_time.feature_importances_
feature_names = ['Current event time','Prev event time', 'Prev event', '2nd prev event', 'Current event']

# sort the features by importance in descending order
indices = np.argsort(importances)[::-1]
sorted_importances = importances[indices]

plt.figure(figsize=(10,6))
plt.bar(feature_names, sorted_importances, color=['brown', 'green', 'blue', 'yellow'])
plt.title('Feature Importances of Random Forest time estimator')
plt.xlabel('Features')
plt.xticks(rotation=0)
plt.ylabel('Importance')
plt.show()

In [ ]:
residual = pd.concat([y_test, pd.Series(y_pred_time)], axis="columns", ignore_index=True)
residual.columns = ['none','real','pred']
residual['error']= residual['real']-residual['pred']

sns.residplot(x='real', y='error', data=residual)
